In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pickle
from collections import defaultdict

from torch.utils.data import Dataset, DataLoader
from Mmetrics import *

import LTR
import datautil
import permutationgraph
import DTR
import EEL
import PPG
import PL

def df2ds(df_path):
    with open(df_path, 'rb') as f:
        df = pickle.load(f)
    ds = df.to_dict(orient='list')
    for k in ds:
        ds[k] = np.array(ds[k])
    ds['dlr'] = np.concatenate([np.zeros(1), np.where(np.diff(ds['qid'])==1)[0]+1, np.array([ds['qid'].shape[0]])]).astype(int)
    return type('ltr', (object,), ds)


def dict2ds(df_path):
    with open(df_path, 'rb') as f:
        ds = pickle.load(f)
    return type('ltr', (object,), ds)

ds2019 = df2ds('LTR2019.df')
ds2020 = df2ds('LTR2020.df')
sds2019 = dict2ds('s_LTR2019.df')
sds2020 = dict2ds('s_LTR2020.df')

In [3]:
def normalize_y(name):
    ds = df2ds(f'LTR{name}.df')
    for qid in range(ds.dlr.shape[0] - 1):
        s, e = ds.dlr[qid: qid+2]
        y = EEL.linspan(ds.y_pred[s:e], 5)
        ds.y_pred[s:e] = y
    with open(f'nLTR{name}.df', 'wb') as f:
        df = pd.DataFrame({'qid':list(ds.qid), 'g':list(ds.g), 'lv':list(ds.lv), 'y_pred':list(ds.y_pred)})
        pickle.dump(df, f)

normalize_y('2019')
normalize_y('2020')

In [10]:
def normalize_y_MSLR():
    with open('p_MSLR_qs.df', 'rb') as f:
        df = pickle.load(f)
    for qid in range(df['dlr'].shape[0] - 1):
        s, e = df['dlr'][qid: qid+2]
        y = EEL.linspan(df['y_pred'][s:e], 5)
        df['y_pred'][s:e] = y
    with open(f'nLTRMSLR_dict.df', 'wb') as f:
        pickle.dump(df, f)

normalize_y_MSLR()

In [6]:
ds = dict2ds(f'p_MSLR_qs.df')
ds.dlr.shape

(1733,)

In [11]:
ds2 = dict2ds(f'nLTRMSLR_dict.df')
ds2.dlr.shape


(1733,)

In [9]:
qid = ds.qid
qid2 = ds2.qid
print(qid.max(), qid2.max())

6111 6111


In [29]:
def subsample(ds, percent):
    subsample_size = int((ds.dlr.shape[0]-1) * percent)
    qids = np.random.choice(np.arange(ds.dlr.shape[0]-1), subsample_size)
#     print(qids)
    sds = {'lv':[], 'y_pred':[], 'qid':[], 'g':[], 'dlr':[0]}
    for qid in qids:
        s,e=ds.dlr[qid:qid+2]
#         print(s,e)
        sds['lv'].append(ds.lv[s:e])
        sds['y_pred'].append(ds.y_pred[s:e])
        sds['g'].append(ds.g[s:e])
        sds['qid'].append(ds.qid[s:e])
        sds['dlr'].append(sds['dlr'][-1]+e-s)
    for k in ['lv', 'y_pred', 'g', 'qid']:
        sds[k] = np.concatenate(sds[k])
    sds['dlr'] = np.array(sds['dlr'])
    return sds

sds2019 = subsample(ds2019, 0.1)
sds2020 = subsample(ds2020, 0.1)


In [32]:
with open('s_LTR2019.df', 'wb') as f:
    pickle.dump(sds2019, f)
    
with open('s_LTR2020.df', 'wb') as f:
    pickle.dump(sds2020, f)

In [12]:
ds2019.lv

array([1., 0., 0., ..., 1., 1., 1.])

In [ ]:

def learn_one_PPG(qid, verbose, y_pred, g, dlr, epochs, lr, exposure, grade_levels, samples_cnt, sessions_cnt):
    s, e = dlr[qid:qid+2]
    y_pred_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=y_pred[s:e], g=g[s:e], sorted_docs=y_pred[s:e].argsort()[::-1], sessions=sessions_cnt)
    objective_ins = EEL.EEL(y_pred = y_pred_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = grade_levels)
    learner = PPG.Learner(  PPG_mat=None, samples_cnt=samples_cnt, 
                                objective_ins=objective_ins, 
                                sorted_docs = sorted_docs_s, 
                                dlr = dlr_s,
#                                 intra = np.arange(g_s.shape[0]),
                                intra = g_s,
                                inter = np.repeat(dlr_s[:-1], np.diff(dlr_s)))
    vals = learner.fit(epochs, lr, verbose=verbose)
    return vals

def learn_all_PPG(y_pred, g, dlr, epochs, lr, exposure, grade_levels, samples_cnt, sessions_cnt):
    sorted_docs = []
    
    for qid in trange(dlr.shape[0] - 1, leave=False):
#     for qid in range(dlr.shape[0] - 1):
        min_b = learn_one_PPG(qid, 0, y_pred, g, dlr, epochs, lr, exposure, grade_levels, samples_cnt, sessions_cnt)
        sorted_docs.append(min_b)
        

    # print(ndcg_dtr(exposure, lv, np.concatenate(y_rerank), dlr, g, query_counts))
    return sorted_docs



In [ ]:
import time
epochs = 50

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])

for qid in range(8):
        start = time.time()
        learn_one_PPG(qid,0, y_pred2020, ds2020.teg, ds2020.tedlr, epochs, 0.1, exposure=exposure2020,
                grade_levels=5, samples_cnt=32, sessions_cnt=20)
        print(qid, 'len:', ds2020.tedlr[qid+1]-ds2020.tedlr[qid], 'took:', time.time()-start)

In [ ]:

for qid in range(8):
        start = time.time()
        learn_one_PPG(qid,0, y_pred2020, ds2020.teg, ds2020.tedlr, epochs, 0.1, exposure=exposure2020,
                grade_levels=5, samples_cnt=32, sessions_cnt=20)
        print(qid, 'len:', ds2020.tedlr[qid+1]-ds2020.tedlr[qid], 'took:', time.time()-start)

In [ ]:

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])
res = learn_all_PPG(y_pred2020, ds2020.teg, ds2020.tedlr, epochs, 0.1, exposure=exposure2020,
        grade_levels=5, samples_cnt=32, sessions_cnt=20)

In [2]:
def evaluate_one(metric, qid, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    s, e = dlr[qid:qid+2]
    permutation = output_permutation[qid]
    lv_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=lv[s:e], g=g[s:e], sorted_docs=lv[s:e].argsort()[::-1], sessions=sessions_cnt)
    
    if metric == 'EEL':
        objective_ins = EEL.EEL(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = 2)
    else:
        objective_ins = DTR.DTR(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure)
        
    
    osl = e - s
    argsort = lv[s:e].argsort()[::-1]
    idcg = ((2.**lv[s:e][argsort][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum()
    ndcg = 0
    for i in range(sessions_cnt):
        ndcg += ((2.**lv[s:e][permutation[i*osl:(i+1)*osl]-(i*osl)][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum() / idcg
        
    return objective_ins.eval(permutation), ndcg / sessions_cnt
 
def evaluate_all(metric, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    eel_res, eer_res, eed_res, ndcgs = [], [], [], []
    for qid in range(dlr.shape[0] - 1):
        s,e = dlr[qid:qid+2]
        if len(np.unique(g[s:e])) == 1:
            continue
        out1, ndcg = evaluate_one(metric, qid, lv, g, dlr, output_permutation, exposure, sessions_cnt)
#         eel, eer, eed = out1
        eel = out1
        eel_res.append(eel)
#         eer_res.append(eer)
#         eed_res.append(eed)
        ndcgs.append(ndcg)
    return np.array(eel_res), np.array(ndcgs)
#     return np.array(eel_res), np.array(eer_res), np.array(eed_res), np.array(ndcgs)

def estimated_evaluate_one(qid, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    s, e = dlr[qid:qid+2]
    permutation = output_permutation[qid]
    lv_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=lv[s:e], g=g[s:e], sorted_docs=lv[s:e].argsort()[::-1], sessions=sessions_cnt)
    objective_ins = EEL.EEL(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = 5)
    return objective_ins.eval_detailed(permutation)
 
def estimated_evaluate_all(lv, g, dlr, output_permutation, exposure, sessions_cnt):
    eel_res, eer_res, eed_res = [], [], []
    for qid in range(dlr.shape[0] - 1):
        s,e = ds.tedlr[qid:qid+2]
        if len(np.unique(g[s:e])) == 1:
            continue
        eel, eer, eed = estimated_evaluate_one(qid, lv, g, dlr, output_permutation, exposure, sessions_cnt)
        eel_res.append(eel)
        eer_res.append(eer)
        eed_res.append(eed)
    return np.array(eel_res), np.array(eer_res), np.array(eed_res)

In [3]:
import pickle
import os

def read_results(directory = '/ivi/ilps/personal/avardas/_data/PPG'):
    files = os.listdir(directory)
    res = {}
    for file in files:
        if 'pkl' not in file:
            continue
        with open(f'{directory}/{file}', 'rb') as f:
            res[file[:-12]] = pickle.load(f)
    return res
    
res = read_results(directory = '/ivi/ilps/personal/avardas/_data/PPG/test')
# res_approx = read_results(directory = '/ivi/ilps/personal/avardas/_data/PPG/test/15')

In [4]:


from tqdm.notebook import tqdm

def get_df_from_results_old(res):
    exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])
    exposure2019 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2019.tedlr).max()+2)])


    dfl = []
    for alg in tqdm(res, leave=True):
        _res = res[alg]
        alg_params = alg.split('_')
    #     print(alg, '-'*30)
        sessions_cnt = int(alg_params[3])
        metric = alg_params[4]
        learner = alg_params[1] + '_' + alg_params[0]
        samples = int(alg_params[2])

        for key in _res:
            if '2019' in key:
                year = 2019
                ds = ds2019
                ypred = y_pred2019
                exposure = exposure2019
            else:
                year = 2020
                ds = ds2020
                ypred = y_pred2020
                exposure = exposure2020

            lr = key.split('_')[1]
            eel_res, ndcg = evaluate_all(metric, ds.telv, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=sessions_cnt)
            dfl.append({'year':year, 'metric':metric, 'learner':learner, 
            'samples':samples, 'sessions':sessions_cnt, 'lr':lr, 'Fairness':eel_res.mean(), 'NDCG':ndcg.mean()})

    return pd.DataFrame(dfl)



def get_df_from_results(res):
    exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.dlr).max()+2)])
    exposure2019 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2019.dlr).max()+2)])


    dfl = []
    for alg in tqdm(res, leave=True):
        _res = res[alg]
        alg_params = alg.split('_')
    #     print(alg, '-'*30)
        sessions_cnt = int(alg_params[2])
        metric = alg_params[3]
        learner = alg_params[1] + '_' + alg_params[0]

        for key in _res:
            if '2019' in key:
                year = 2019
                ds = ds2019
                exposure = exposure2019
            else:
                year = 2020
                ds = ds2020
                exposure = exposure2020

            lr = key.split('_')[1]
            samples = int(key.split('_')[2])
            eel_res, ndcg = evaluate_all(metric, ds.lv, ds.g, ds.dlr, _res[key], exposure, sessions_cnt=sessions_cnt)
            dfl.append({'year':year, 'metric':metric, 'learner':learner, 
            'samples':samples, 'sessions':sessions_cnt, 'lr':lr, 'Fairness':eel_res.mean(), 'NDCG':ndcg.mean()})

    return pd.DataFrame(dfl)


df = get_df_from_results(res)

In [9]:
print(df.sort_values(by=['metric', 'sessions', 'year', 'learner']).head(100))

    year metric      learner  samples  sessions    lr  Fairness      NDCG
48  2019    DTR   PL_nointra       32         1  0.01  0.603934  0.761296
29  2019    DTR    PPG_intra       32         1  0.01  0.564656  0.793540
25  2019    DTR  PPG_nointra       32         1  0.01  0.565660  0.764338
47  2020    DTR   PL_nointra        8         1  0.01  1.182406  0.348552
28  2020    DTR    PPG_intra       32         1  0.01  1.117481  0.391401
..   ...    ...          ...      ...       ...   ...       ...       ...
4   2020    EEL   PL_nointra       32        16  0.01  0.096880  0.356623
30  2020    EEL    PPG_intra       16        16  0.01  0.150840  0.389316
34  2020    EEL  PPG_nointra       32        16  0.01  0.109646  0.357069
33  2019    EEL   PL_nointra       32        32  0.01  0.028359  0.757571
32  2020    EEL   PL_nointra       32        32  0.01  0.101208  0.358215

[65 rows x 8 columns]


In [11]:
def refine_df(df, metric, year):
    return df.loc[(df.metric==metric) & (df.year == year), sorted(list(set(df.columns)-set(('metric', 'year'))))].sort_values(by=['sessions', 'learner'])

print()
print(refine_df(df, 'EEL', 2019).head(100))
print(refine_df(df, 'EEL', 2020).head(100))
print(refine_df(df, 'DTR', 2019).head(100))
print(refine_df(df, 'DTR', 2020).head(100))

    Fairness      NDCG      learner    lr  samples  sessions
38  0.059956  0.752894   PL_nointra  0.01       32         1
21  0.046645  0.799777    PPG_intra  0.01        8         1
17  0.043765  0.766477  PPG_nointra  0.01        4         1
60  0.044002  0.748923   PL_nointra  0.01       16         2
15  0.044671  0.797066    PPG_intra  0.01        8         2
11  0.043007  0.755952  PPG_nointra  0.01        8         2
1   0.036602  0.758194   PL_nointra  0.01       32         4
56  0.040269  0.794528    PPG_intra  0.01        4         4
58  0.036585  0.751694  PPG_nointra  0.01        4         4
19  0.032408  0.756587   PL_nointra  0.01       16         8
40  0.038225  0.792841    PPG_intra  0.01        4         8
36  0.035488  0.756497  PPG_nointra  0.01        8         8
5   0.029446  0.760406   PL_nointra  0.01       32        16
31  0.036070  0.792974    PPG_intra  0.01        4        16
33  0.028359  0.757571   PL_nointra  0.01       32        32
    Fairness      NDCG  

In [ ]:
e

In [ ]:

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])
exposure2019 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2019.tedlr).max()+2)])


for alg in res:
    _res = res[alg]
    print(alg, '-'*30)
    sessions_cnt = int(alg.split('_')[3])
    metric = alg.split('_')[4]
    for key in _res:
        if '2019' in key:
            ds = ds2019
            ypred = y_pred2019
            exposure = exposure2019
        else:
            ds = ds2020
            ypred = y_pred2020
            exposure = exposure2020
            
#         eel_res, eer_res, eed_res, ndcg = evaluate_all(ds.telv, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=sessions_cnt)
        eel_res, ndcg = evaluate_all(metric, ds.telv, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=sessions_cnt)
#         es_eel_res, es_eer_res, es_eed_res = estimated_evaluate_all(ypred, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=20)
    
        print(f'{key}', eel_res.mean(), '(', ndcg.mean(), ')')
#         print(f'{key}', eel_res.mean(), '(', es_eel_res.mean(), ')')
#         print(f'\t', ndcg.mean())
#         print(f'\t', eer_res.mean(), '(', es_eer_res.mean(), ')')
#         print(f'\t', eed_res.mean(), '(', es_eed_res.mean(), ')')


In [ ]:
def test(qid, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    s, e = dlr[qid:qid+2]
    permutation = output_permutation[qid]
    lv_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=lv[s:e], g=g[s:e], sorted_docs=lv[s:e].argsort()[::-1], sessions=sessions_cnt)
    objective_ins = EEL.EEL(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = 2)
    
    osl = e - s
    argsort = lv[s:e].argsort()[::-1]
    print(argsort)
    print(lv[s:e])
    idcg = ((2.**lv[s:e][argsort][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum()
    print('idcg:', idcg)
    ndcg = 0
    for i in range(sessions_cnt):
        dcg = ((2.**lv[s:e][permutation[i*osl:(i+1)*osl]-(i*osl)][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum()
        print('dcg:', dcg)
        ndcg += dcg / idcg
        
    return objective_ins.eval_detailed(permutation), ndcg / sessions_cnt

In [ ]:
# ds = ds2019
# exposure = exposure2020
# _res = res['PPG']
# key = '2019_0.01'
test(1,ds.telv, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=20)

In [ ]:
eel_res, eer_res, eed_res, ndcg = evaluate_all(ds.telv, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=20)

In [ ]:
ndcg